# Projet floutage de visage

Dans la section qui suit on ajoute tous les import necéssaires au programme. <br/>
cv2 pour OpenCV nous sert dans la capture vidéo et dans le floutage du visage. <br/>
numpy nous sert dans l'une des deux versions de floutage. <br/>
skimage pour les filtres qui servent à la détection de visage. <br/>

In [1]:
import cv2
import numpy as np
import skimage
from skimage import filters
from skimage.feature import Cascade

On initialise la capture de vidéo et le détecteur de visage, on les utilisera par la suite.

In [2]:
cap = cv2.VideoCapture(0)

trained = skimage.data.lbp_frontal_face_cascade_filename()
detector = Cascade(trained)

<p>On vient ensuite faire une capture vidéo, toute les 5 images capturées on vient appliquer un traitement dessus. <br/>
On récupère l'image en question, on vient lui appliquer un filtre pour la débruiter, ce n'est qu'après cela que l'on applique la détection de visage. <br/>
Pour chaque visage trouvé on vient remplacer cette portion d'image par une copie d'elle même à laquelle on applique un floutage. </p>
<p>On vient ensuite afficher l'image ainsi modifiée pour appliquer ce floutage en temps réel. Cette méthode fonctionne parfaitement bien sur des visages d'une certaine dimension. Le problème étant que si un visage est trop éloigné, ou inversement si il est trop proche, alors la détection ne s'appliquera pas. Si nous mettons une trop grande plage de détection à notre programme, il va essayer de détecter des visages même minuscules, ce qui cause de gros problèmes de performances.</p>

In [3]:
cpt = 0
while True:
    cpt += 1
    if cpt%5 == 0: # Traitement tout les 5 frames
        # Capture de la frame
        ret, frame = cap.read()

        # Débruitage de l'image
        filtered_frame = filters.median(frame)

        # Detection des visages
        detected = detector.detect_multi_scale(filtered_frame,
                                               scale_factor=1.2,
                                               step_ratio=1,
                                               min_size=(90, 90),
                                               max_size=(200, 200))


        # Pour tout les visages detect
        for patch in detected:
            r = patch['r']
            c = patch['c']
            width = patch['width']
            height = patch['height']

            # get the rectangle img around all the faces
            face = filtered_frame[r:r+height, c:c+width]
            
            # Floutage de l'image
            # option 1 : avec filter2D
            #kernel = np.ones((10, 10), np.float32) / 100
            #blur_face = cv2.filter2D(face, -1, kernel)

            #option 2 : avec un filtre gaussien
            blur_face = cv2.GaussianBlur(face, (15, 15), 8)
            # merge this blurry rectangle to our final image
            filtered_frame[r:r+height, c:c+width] = blur_face

        # Display the resulting frame
        cv2.imshow('floutage visage',filtered_frame)

    # Quitter la fenetre lors de l'appui sur "q"
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

On obtient une capture vidéo qui détecte les visages et vient appliquer un flou par dessus. <br/>
Pour quitter la fenêtre il faut appuyer sur la touche 'q'.

In [4]:
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

Une fois le programme terminé on relache la capture et on détruit la fenêtre associée.